<a href="https://colab.research.google.com/github/gitdhirajsv/Awesome-Quant-Machine-Learning-Trading/blob/master/GPTPythonNotes57b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
olgabelitskaya_python_recipes_path = kagglehub.dataset_download('olgabelitskaya/python-recipes')

print('Data source import complete.')


# 57b. Introduction to data science.
# Data manipulations with Python modules

# 57b. Introducción a la ciencia de datos.
# Manipulaciones de datos con módulos de Python

- [NumPy documentation](https://numpy.org/doc/stable/)
- [pandas documentation](https://pandas.pydata.org/docs/)
- [polars API reference](https://pola-rs.github.io/polars/py-polars/html/reference/index.html)
- [datatable User Guide](https://datatable.readthedocs.io/en/latest/manual/index-manual.html#user-guide)
- [modin Usage Guide](https://modin.readthedocs.io/en/stable/usage_guide/index.html)
- [Vaex docs](https://vaex.io/docs/index.html)
- [CuPy](https://docs.cupy.dev/en/stable/)

In [ ]:
import polars as po, pandas as pd, numpy as np
import random, datetime, pylab as pl, plotly.express as px
po.show_versions()

In [ ]:
%%writefile example.csv
id,name,position,department,age,gender,salary,start_date,email,phone
f01,John Smith,HR Manager,Human Resources,35,Male,60000,2015-03-15,john.smith@example.com,123-456-7890
f02,Emily Johnson,Marketing Specialist,Marketing,28,Female,45000,2017-09-22,emily.johnson@example.com,234-567-8901
f03,Michael Williams,Software Engineer,Engineering,30,Male,75000,2018-01-10,michael.williams@example.com,345-678-9012
f04,Sophia Brown,Financial Analyst,Finance,32,Female,58000,2016-05-18,sophia.brown@example.com,456-789-0123
f05,William Jones,Sales Representative,Sales,29,Male,48000,2019-11-04,william.jones@example.com,567-890-1234
f06,Olivia Davis,Administrative Assistant,Admin,26,Female,38000,2020-07-12,olivia.davis@example.com,678-901-2345
f07,Alexander Miller,Marketing Manager,Marketing,38,Male,68000,2014-02-28,alexander.miller@example.com,789-012-3456
f08,Mia Wilson,Network Administrator,IT,33,Female,65000,2017-08-05,mia.wilson@example.com,890-123-4567
f09,Daniel Taylor,Accountant,Finance,31,Male,55000,2019-03-25,daniel.taylor@example.com,901-234-5678
f10,Isabella Moore,Project Manager,Project Management,34,Female,70000,2016-11-15,isabella.moore@example.com,012-345-6789
f11,Ethan Anderson,Software Developer,Engineering,27,Male,60000,2018-04-30,ethan.anderson@example.com,123-456-7890
f12,Chloe Lee,HR Coordinator,Human Resources,29,Female,42000,2020-06-08,chloe.lee@example.com,234-567-8901
f13,James Garcia,Sales Manager,Sales,37,Male,72000,2015-09-14,james.garcia@example.com,345-678-9012
f14,Natalie Rodriguez,Data Analyst,IT,31,Female,56000,2017-02-12,natalie.rodriguez@example.com,456-789-0123
f15,Andrew Martinez,Operations Manager,Operations,36,Male,69000,2016-03-20,andrew.martinez@example.com,567-890-1234
f16,Grace Hernandez,Executive Assistant,Admin,27,Female,45000,2019-08-10,grace.hernandez@example.com,678-901-2345
f17,Lucas Gonzales,Quality Assurance Specialist,Engineering,29,Male,52000,2018-12-04,lucas.gonzales@example.com,789-012-3456
f18,Avery Perez,Financial Manager,Finance,40,Female,80000,2013-06-18,avery.perez@example.com,890-123-4567
f19,Henry Turner,Customer Service Representative,Customer Service,25,Male,40000,2020-11-30,henry.turner@example.com,901-234-5678
f20,Samantha Scott,Marketing Coordinator,Marketing,31,Female,48000,2017-04-15,samantha.scott@example.com,012-345-6789

## `polars` & `pandas`

In [ ]:
example_po = po.read_csv('example.csv',try_parse_dates=True)\
.with_columns(po.col('age').cast(po.Int32))
example_po.head(3)

In [ ]:
example_pd = pd.read_csv('example.csv', parse_dates=True)
example_pd['age'] = example_pd['age'].astype('int32')
display(example_pd.head(3))
example_pd.info()

In [ ]:
sample_po = po.scan_csv("example.csv")\
.select(["name", "age", "gender", "salary"])\
.filter(po.col("age") > 30).fetch(10)
sample_po

In [ ]:
sample_pd = pd.read_csv("example.csv")[["name", "age", "gender", "salary"]]
sample_pd = sample_pd[sample_pd['age'] > 30].head(10)
sample_pd

In [ ]:
example_dict = {"wOrDs_ID": [1, 2, 3, 4, 5],
                "SenTEnCe": ["this", "is", "hard", "to", "read"]}
example_dict_po = po.DataFrame(example_dict)
example_dict_po.write_csv('example_dict.csv')
po.scan_csv(
    'example_dict.csv',
    with_column_names=lambda cols: [col.lower() for col in cols]
).collect()

In [ ]:
example_dict_pd = pd.DataFrame(example_dict)
example_dict_pd.to_csv('example_dict.csv', index=False)
example_dict_pd = pd.read_csv(
    'example_dict.csv', header=0,
    names=[col.lower() for col in example_dict_pd.columns])
example_dict_pd

Objects of `polars` || Objetos de `polars`

In [ ]:
# this function creates random examples
# esta función crea ejemplos aleatorios
randi = {
    'int': lambda: random.randint(0, 100),
    'float': lambda: random.uniform(0, 100),
    'chr': lambda: random.choices('abcdefghijklmnopqrstuvwxyz')[0],
    'int_list': lambda: random.sample(range(100), random.randint(1, 10)),
    'int_tuple': lambda: tuple(random.sample(range(100), random.randint(1, 10))),
    'float_list': lambda: [round(random.uniform(0, 100), 2)
                           for _ in range(random.randint(1, 10))],
    'bool': lambda: random.choice([True, False]),
    'none': lambda: None,
    'date': lambda: str(datetime.date(1970, 1, 1) +
                        datetime.timedelta(random.randint(1, 60*365))),
    'numpy_nan': lambda: np.nan,
    'numpy_int64': lambda: np.int64(random.randint(0, 100)),
    'numpy_float64': lambda: np.float64(random.uniform(0, 100)),
    'polars_null': lambda: po.Null,
    'polars_unknown': lambda: po.Unknown,
}
def random_list(start=3, end=10, random_dict=randi, seed=12, choice=None):
    random.seed(seed)
    if choice == None:
        choice = list(random_dict.keys())
    elif not isinstance(choice, list):
        choice = [choice]
    return [random_dict[random.choice(choice)]()
            for _ in range(random.randint(start, end))]

`1D - Series`

In [ ]:
s_obj = po.Series("random types", random_list(start=4, end=12, seed=12),
                  dtype=po.Object)
s_lst_int = po.Series("random lists",
                  random_list(start=4, end=5, seed=15, choice=['int_list', 'float_list']))
s_lst_float = po.Series("random lists",
                  random_list(start=4, end=5, seed=13, choice=['int_list', 'float_list']))
display(s_obj, s_lst_int, s_lst_float)

In [ ]:
s_lst = s_lst_int.cast(po.List(po.Float64)).append(s_lst_float)
s_lst, s_lst.n_chunks()

`2D - DataFrames`

In [ ]:
# they will be columns of different lengths in the resulting data frames
# serán columnas de diferentes longitudes en los marcos de datos resultantes
d = {'A': random_list(start=4, end=12, seed=14, choice='int_list'),
     'B': random_list(start=4, end=12, seed=15, choice='int_list')}

In [ ]:
po.concat(items=[po.DataFrame({k: v}) for k, v in d.items()], how="horizontal")

In [ ]:
pd.DataFrame.from_dict(d, orient='index').T

`2D - LazyFrames`

In [ ]:
# chain operations without materializing intermediate results
# operaciones en cadena sin materializar resultados intermedios
lf = po.LazyFrame(
     np.array([(1, 2, 1), (3, 4, 5), (6, 7, 6)], dtype=np.float32),
     schema=["A", "B", "C"], orient="col")
lf.filter(po.col("A") < 2)\
.select([po.col("A"), po.col("B")])\
.sort("B").collect()

`Expr - Expressions`

In [ ]:
po.Series(["2020-01-01 01:00Z", "2020-01-01 02:00Z"])\
.str.strptime(po.Datetime, "%Y-%m-%d %H:%M%#z")

In [ ]:
weather_dict = {
    "station": ["station_" + str(x) for x in range(1, 6)],
    "temperature": ["20 5 5 E1 7 13 19 9 6 20",
                    "18 8 16 11 23 E2 8 E2 E2 E2 90 70 40",
                    "19 24 E9 16 6 12 10 22",
                    "E2 E0 15 7 8 10 E1 24 17 13 6",
                    "14 8 E0 16 22 24 E1"]}
weather_df_po = po.DataFrame(weather_dict)
weather_df_po.with_columns(po.col("temperature").str.split(" ")\
.list.eval(po.element().cast(po.Int64, strict=False)))\
.with_columns(po.col("temperature").list.head(3).alias("top_3"),
              po.col("temperature").list.tail(3).alias("bottom_3"),
              po.col("temperature").list.len().alias("amount"),
              po.col("temperature").list.eval(
                  po.element().is_null()).list.sum().alias("errors"))

`SQL`

In [ ]:
example_po_lazy = example_po.lazy()
ctx = po.SQLContext(example_po_lazy=example_po_lazy)
from sqlite3 import connect
conn = connect(':memory:')
example_pd.to_sql(name='example_pd', con=conn)
display(example_po_lazy.head(1).collect())
type(example_po), type(example_po_lazy), type(example_pd)

In [ ]:
queries_po = ("SELECT * FROM example_po_lazy WHERE department = 'IT';",
              "SELECT department, AVG(salary) AS average_salary "
              "FROM example_po_lazy GROUP BY department;",
              "SELECT position, COUNT(*) AS num_employees "
              "FROM example_po_lazy GROUP BY position;",
              "SELECT MAX(salary) AS highest_salary FROM example_po_lazy;",
              "SELECT name, email FROM example_po_lazy WHERE salary > 70000;",)
for q in queries_po:
    display(ctx.execute(q).collect())

In [ ]:
queries_po2 = ("example_po_lazy.filter(po.col('department') == 'IT')",
               "example_po_lazy.group_by('department').agg(po.col('salary')"
               ".mean()).rename({'salary': 'average_salary'})",
               "example_po_lazy.group_by('position').count()"
               ".rename({'count': 'num_employees'})",
               "example_po_lazy.group_by('position')."
               "agg(po.col('position').count().alias('num_employees'))",
               "example_po_lazy.select(po.col('salary')).max()",
               "example_po_lazy.filter(po.col('salary') > 70000)"
               ".select([po.col('name'), po.col('email')])")
for q in queries_po2:
    display(eval(q + ".collect()")); print(120*'_')

In [ ]:
queries_pd = [q.replace('example_po_lazy', 'example_pd') for q in queries_po]
for q in queries_pd:
    display(pd.read_sql(q, conn))

In [ ]:
queries_pd2 = ("example_pd[example_pd['department'] == 'IT']",
               "example_pd.groupby('department')['salary'].mean()",
               "example_pd['position'].value_counts()",
               "example_pd['salary'].max()",
               "example_pd[example_pd['salary'] > 70000][['name', 'email']]")
for q in queries_pd2:
    display(eval(q)); print(120*'_')

`polars => plotly & matplotlib`

In [ ]:
d = {'A': sorted(random_list(start=100, end=101, seed=14, choice='float')),
     'B': random_list(start=100, end=101, seed=15, choice='float')}
df_po = po.DataFrame(d)
px.line(x=df_po["A"], y=df_po["B"])

In [ ]:
fig, ax = pl.subplots(figsize=(8, 3))
colours = ['tab:orange', 'tab:green']
for idx, (gender, gender_po) in enumerate(
    example_po.partition_by("gender", as_dict=True).items()):
    color = colours[idx]
    ax.scatter(x=gender_po["age"].log(),
               y=gender_po["salary"].log(),
               c=color, label=gender,)
ax.legend(); pl.grid(color='slategray')

## `datatable`

In [ ]:
!pip install -q datatable

In [ ]:
import datatable as dt, math
from datatable import f, fread, ifelse

In [ ]:
DT1 = dt.Frame(
    A=range(5), B=[1.7, 3.4, 0, None, -math.inf], stypes={"A": dt.int64})
DT2 = dt.Frame(example_pd)
# C0, C1, C2... as column names by default
# C0, C1, C2... como nombres de columnas por defecto
DT3 = dt.Frame(
    np.array([(1, 2, 1), (3, 4, 5), (6, 7, 6)], dtype=np.float32))
display(DT1), print(DT2.names), display(DT3)

| Syntax              | Description                                      | Descripción |
|---------------------|--------------------------------------------------|- -------------------------------------------------|
| `f[:]`               | Select all columns                               |Seleccionar todas las columnas |
| `f[::-1]`            | Select all columns in reverse order              | Seleccionar todas las columnas en orden inverso |
| `f[:5]`              | Select the first 5 columns                       | Seleccione las primeras 5 columnas |
| `f[3:4]`             | Select the fourth column                          | Seleccione la cuarta columna |
| `f["B":"H"]`         | Select columns from B to H, inclusive             | Seleccione columnas de B a H, inclusive |
| `f[int]`             | Select all integer columns                        | Seleccionar todas las columnas de números enteros |
| `f[float]`           | Select all floating-point columns                 | Seleccionar todas las columnas de punto flotante |
| `f[dt.str32]`        | Select all columns with stype str32               | Seleccione todas las columnas con estilo str32 |
| `f[None]`            | Select no columns (empty columnset)               | No seleccionar columnas (conjunto de columnas vacío) |

In [ ]:
# f-expressions => to the columns of a current frame
# expresiones f => para las columnas de un marco actual
ffilter = (f.C1 >= 4)
DT3[ffilter,:]

In [ ]:
fl = (f[int].extend(f[float]),          # integer and floating-point columns
                                        # # columnas de números enteros y de punto flotante
      f[:3].extend(f[-3:]),             # the first and the last 3 columns
                                        # las primeras y las últimas 3 columnas
      f.A.extend(f.B),                  # columns "A" and "B"
                                        # las columnas "A" y "B"
      f[str].extend(dt.str32(f[int])))  # string columns, and also all integer columns
                                        # converted to strings
                                        # columnas de cadenas, y también todas las columnas
                                        # de números enteros convertidas en cadenas
display(DT1[:, fl[2]])
display(DT2[:3, fl[0]])

In [ ]:
pay = """
       ID|Charges|Payment_Method
       634-VHG|28|Cheque
       365-DQC|33.5|Credit card
       264-PPR|631|--
       845-AJO|42.3|
       789-KPO|56.9|Bank Transfer
       """
pay_dt = fread(pay, na_strings=['--', ''])
display(pay_dt)

In [ ]:
example_dt = fread('example.csv')
# SQL: "SELECT * FROM example_dt WHERE department = 'IT';"
display(example_dt[(f.department == 'IT'), :])
# SQL: "SELECT department, AVG(salary) AS average_salary
#       FROM example_dt GROUP BY department;"
display(example_dt[:, {"average_salary": dt.mean(f.salary),},
                   dt.by("department")])

In [ ]:
data = {'name': ['Tomas', 'Sophia', 'Mary', 'Daniel', 'Olivia', 'James', 'Barbara'],
        'age': [30, 41, 32, 67, 44, 52, 47],
        'rank': ['90%', '95%', '100%', '82%', '87%', '86%', '93%'],
        'age_': [30, 41, None, 67, None, 52, 47],
        'income': [10000, 8000, 5500, 6200, 12000, 7300, 11500]}
df1 = dt.Frame(data)
gender = {'Tomas': 'M', 'Sophia': 'F', 'Mary': 'F', 'Daniel': 'M',
          'Olivia': 'F', 'James': 'M', 'Barbara': 'F'}
gender_frame = dt.Frame({'name': list(gender.keys()), 'gender': list(gender.values())})
gender_frame.key = 'name'; df1 = df1[:, :, dt.join(gender_frame)]
surnames = {'Tomas': 'Murray', 'Sophia': 'Evans', 'Mary': 'Taylor', 'Daniel': 'Gilbert',
            'Olivia': 'Williams', 'James': 'Paterson', 'Barbara': 'Harrison'}
surname_frame = dt.Frame({'name': list(surnames.keys()), 'surname': list(surnames.values())})
surname_frame.key = 'name'; df1 = df1[:, :, dt.join(surname_frame)]
mean_income = df1[:, dt.mean(f.income)]
df1[:, dt.update(higher_income=f.income > mean_income)]
cond = ifelse((f.age < 50) & (f.income > 7500), True, False)
df1[:, dt.update(interview = cond)]
fcond = lambda give, amount: ifelse(give, f.income / f.age * amount, 0)
df1[:, dt.update(bonus = fcond(True, 6.5))]
df1[:, dt.update(year_bonus = f.bonus * 4)]
df1

In [ ]:
df1 = pd.DataFrame({
    'name': ['Tomas', 'Sophia', 'Mary', 'Daniel',
             'Olivia', 'James', 'Barbara'],
    'age': [30, 41, 32, 67, 44, 52, 47],
    'rank': ['90%', '95%', '100%', '82%', '87%', '86%', '93%'],
    'age_': [30, 41, None, 67, None, 52, 47],
    'income': [10000, 8000, 5500, 6200, 12000, 7300, 11500]
})
gender = {'Tomas':'M', 'Sophia':'F', 'Mary':"F", 'Daniel':"M",
          'Olivia':"F", 'James':"M", 'Barbara':"F"}
df1['gender'] = df1['name'].map(gender)
surnames = pd.Series(
    ['Murray', 'Evans', 'Taylor', 'Gilbert',
     'Williams', 'Paterson', 'Harrison'], index=df1['name'])
df1['surname'] = df1['name'].map(surnames)
mean_income = df1['income'].mean()
def higher_income(x):
    return x > mean_income
df1['higher_income'] = df1['income'].map(higher_income)
def interview(row):
    return row['age'] < 50 and row['income'] > 7500
df1['interview'] = df1.apply(interview, axis=1)
def bonus(row, amount, give=False):
    if give: return row['income'] / row['age'] * amount
    else: return 0
df1['bonus'] = df1.apply(bonus, args=(6.5,), give=True, axis=1)
df1 = df1.assign(year_bonus=df1['bonus']*4)
df1

In [ ]:
data = {'ID': ["x%d" % r for r in range(10)],
        'gender': ['F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'M'],
        'exam_year': ['2007', '2007', '2007', '2008', '2008',
                      '2008', '2008', '2009', '2009', '2009'],
        'class': ['algebra', 'stats', 'bio', 'algebra', 'algebra',
                  'stats', 'stats', 'algebra', 'bio', 'bio'],
        'participated': ['yes', 'yes', 'yes', 'yes', 'no',
                         'yes', 'yes', 'yes', 'yes', 'yes'],
        'employed': [True, True, True, False, False,
                     False, False, True, True, False],
        'grade': [48, 99, 75, 80, 42, 80, 72, 68, 36, 78]}
df2 = dt.Frame(data)
cond = ifelse(f.grade > 50, 'yes', 'no')
df2[:, dt.update(passed = cond)]
display(df2[:, {"participated": dt.sum(f.participated == "yes"),
                "passed": dt.sum(f.passed == "yes"),
                "employed": dt.sum(f.employed),
                "grade": dt.mean(f.grade)},
            dt.by("exam_year")])
df2 = pd.DataFrame(data)
df2['passed'] = df2.apply(lambda row: row['grade'] > 50, axis=1)
display(df2.groupby('exam_year')\
           .agg({'participated': lambda x: x.value_counts()['yes'],
                 'passed': lambda x: sum(x == 'yes'),
                 'employed': lambda x: sum(x),
                 'grade': lambda x: sum(x) / len(x)}))

`datatable => plotly & matplotlib`

In [ ]:
d = {'A': sorted(random_list(start=100, end=101, seed=14, choice='float')),
     'B': random_list(start=100, end=101, seed=15, choice='float')}
df_dt = dt.Frame(d)
px.line(x=df_dt[:, "A"].to_list()[0], y=df_dt[:, "B"].to_list()[0])

In [ ]:
fig, ax = pl.subplots(figsize=(8, 3))
ax.scatter(example_dt[:, "age"].to_list()[0],
           example_dt[:, "salary"].to_list()[0])
pl.xlabel("Age"); pl.ylabel("Salary")
pl.title("Age vs. Salary"); pl.grid();

## `vaex`

In [ ]:
%pip install -q vaex

In [ ]:
import vaex

In [ ]:
x = random_list(start=25, end=25, seed=16, choice=['int', 'float', 'chr'])
y = random_list(start=25, end=25, seed=13, choice=['int', 'float'])
z = random_list(start=25, end=25, seed=10, choice=['chr'])
df = vaex.from_arrays(x=x, y=y, z=z)
display(df.head(7)); display(df.describe(selection=df.y>25))

In [ ]:
for i1, i2, chunk in df.evaluate_iterator(df.y, chunk_size=5):
    print(f"id from {i1} to {i2-1}: sum y = {chunk.sum()}")

In [ ]:
data = {'name': ['Tomas', 'Sophia', 'Mary', 'Daniel', 'Olivia', 'James', 'Barbara'],
        'age': [30, 41, 32, 67, 44, 52, 47],
        'rank': ['90%', '95%', '100%', '82%', '87%', '86%', '93%'],
        'age_': [30, 41, None, 67, None, 52, 47],
        'income': [10000, 8000, 5500, 6200, 12000, 7300, 11500],
        'gender': list('MFFMFMF')}
df1 = vaex.from_dict(data)
# a virtual column (computed on the fly)
# una columna virtual (calculada sobre la marcha)
df1['bonus'] = 6.5 * df1.income  / df1.age
# turn columns into the native CPU format for optimal performance at memory cost
# convierta las columnas al formato nativo de la CPU para
# obtener un rendimiento óptimo con un coste de memoria
df1 = df1.materialize('bonus')
display(df1); display(df1['bonus'])
@vaex.register_function()
def year_bonus(bonus):
    return bonus * 4
display(df1.bonus.year_bonus())

In [ ]:
@vaex.register_function(as_property=True)
def interview_property(age, income):
    result = []
    for a, i in zip(age.tolist(), income.tolist()):
        result.append((a < 50) & (i > 7500))
    return np.array(result)
# add a visible column / agregar una columna visible
# df1['interview'] = (df1.age < 50) & (df1.income > 7500)
# add a custom property / agregar una propiedad personalizada
df1.interview = df1.func.interview_property(df1.age, df1.income)
display(df1); display(df1.interview);

In [ ]:
x = np.linspace(-1.1, 1.1, 100)
y = np.linspace(-1.1, 1.1, 100)
X, Y = np.meshgrid(x, y)
k = np.array(range(1, 10000, 2))
x0 = np.cos(14*np.pi*k/10**4)*(1-.75*(np.cos(20*np.pi*k/10**4))**2)
y0 = np.sin(14*np.pi*k/10**4)*(1-.75*(np.cos(24*np.pi*k/10**4))**2)
r = 10**(-2)+.1*np.sin(54*np.pi*k/10**4)**6
F = np.array([(X-x0[(i-1)//2])**2+(Y-y0[(i-1)//2])**2-r[(i-1)//2]**2
              for i in k])
df_va = vaex.from_arrays(k=k, x0=x0, y0=y0, r=r, F=F)
pl.figure(figsize=(8, 3))
df_va.viz.histogram(df_va.r, limits='75.%');
pl.figure(figsize=(8, 3))
df_va.viz.heatmap(
    [["x0", "y0"]], what=["count(*)", "mean(r)"],
    selection=["-1.2 < r < 1.2"], limits='99%');
fig = pl.figure(figsize=(8, 8))
ax = fig.add_subplot()
for i in range(5000):
    ax.contour(X, Y, F[i], [0],
               colors=[pl.cm.hot(i/5000)[:3]], linewidths=.1);
ax.set_xlim((-1.1, 1.1)); ax.set_ylim((-1.1, 1.1));

In [ ]:
# huge datasets
taxi = vaex.open('s3://vaex/taxi/yellow_taxi_2015_f32s.hdf5?anon=true')
taxi.shape

In [ ]:
taxi['fare_by_distance'] = (taxi.fare_amount / taxi.trip_distance)
taxi[['pickup_datetime','dropoff_datetime','fare_by_distance']].head(3)

In [ ]:
taxi[taxi.trip_distance>0].fare_by_distance.mean(progress='widget')

In [ ]:
counts_geo = taxi.count(
    binby=[taxi.pickup_longitude, taxi.pickup_latitude],
    shape=256, limits=[[-74.37, -73.54], [40.49, 40.92]], progress='widget')

In [ ]:
pl.figure(figsize=(8, 8))
pl.imshow(np.log1p(counts_geo).T, origin='lower', cmap='hot', aspect='auto');

## `CuPy`

In [ ]:
import cupy as cp

In [ ]:
x_cpu, x_gpu = np.array([1, 2, 3]), cp.array([1, 2, 3])
y_cpu, y_gpu = np.array([4, 5, 6]), cp.array([4, 5, 6])
(x_cpu + y_cpu, x_gpu + y_gpu,
 x_cpu + cp.asnumpy(y_gpu), x_gpu + cp.asarray(y_cpu))

In [ ]:
(np.array([-1], dtype=np.float32).astype(np.uint32),
 cp.array([-1], dtype=np.float32).astype(np.uint32),
 np.array([float('inf')], dtype=np.float32).astype(np.int32),
 cp.array([float('inf')], dtype=np.float32).astype(np.int32))

In [ ]:
np_rand = np.array(np.random.randn())
cp_rand = cp.random.randn(dtype=np.float32)
np_rand.dtype, cp_rand

In [ ]:
# in progress ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ